# Bixi Data Mining Project

Students: Quan Hao, 11248609; Gabriel Lainesse, 11189782; Chaoyang Zheng, 11249259 

Course: Data Mining Techniques

# Loading required libraries

In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sklearn
import tensorflow as tf
from tensorflow import keras
import patsy
import seaborn as sns
import graphviz
%matplotlib inline

# Importing the data

In [36]:
hdf_file = r"/Users/gabriel/dataset_2017.hdf"

In [37]:
dataset = pd.read_hdf(path_or_buf=hdf_file,key="dataset")

In [38]:
# Checking dataset content and datatypes
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4740357 entries, 0 to 4740356
Data columns (total 52 columns):
start_date                datetime64[ns]
end_date                  datetime64[ns]
duration_sec              int64
is_member                 int64
start_code                int64
start_name                object
start_latitude            float64
start_longitude           float64
start_neighborhood        object
start_great_park          object
start_affectation         object
end_code                  int64
end_name                  object
end_latitude              float64
end_longitude             float64
end_neighborhood          object
end_great_park            object
end_affectation           object
start_date_hour           int64
end_date_hour             int64
Weather Condition         object
Humidity                  float64
Pressure                  float64
Wind Speed                float64
Temperature               float64
festivial_name_1          object
festival1_l

# Data Pre-Processing for Model Building

Further data pre-processing is required in order to build models

### One-hot encoding of weather conditions

In [39]:
# Checking Weather condition values
dataset['Weather Condition'].value_counts()

broken clouds                   1383352
few clouds                      1011042
overcast clouds                  648132
scattered clouds                 482542
mist                             341285
sky is clear                     332341
light intensity shower rain      163973
light rain                       127779
fog                               84218
proximity shower rain             81129
thunderstorm with light rain      23908
haze                              16603
proximity thunderstorm            11421
thunderstorm                       8823
light intensity drizzle            7795
thunderstorm with rain             5089
moderate rain                      3859
light shower snow                  2380
light intensity drizzle rain       2286
thunderstorm with heavy rain       2038
shower rain                         362
Name: Weather Condition, dtype: int64

In [40]:
# One-hot encoding of weather conditions
# Group up the weather categories that could belong together (such as "thunderstorm with light rain" and "thunderstorm with heavy rain")
dataset['Weather_Rain'] = dataset['Weather Condition'].str.contains('rain').astype(bool)
dataset['Weather_Thunderstorm'] = dataset['Weather Condition'].str.contains('thunderstorm').astype(bool)
dataset['Weather_Fog'] = dataset['Weather Condition'].str.contains('fog').astype(bool)
dataset['Weather_Mist'] = dataset['Weather Condition'].str.contains('mist').astype(bool)
dataset['Weather_Snow'] = dataset['Weather Condition'].str.contains('snow').astype(bool)
dataset['Weather_Drizzle'] = dataset['Weather Condition'].str.contains('drizzle').astype(bool)
dataset['Weather_Haze'] = dataset['Weather Condition'].str.contains('haze').astype(bool)
dataset['Weather_Clear'] = dataset['Weather Condition'].str.contains('clear').astype(bool)
dataset['Weather_BrokenClouds'] = dataset['Weather Condition'].str.contains('broken clouds').astype(bool)
dataset['Weather_FewClouds'] = dataset['Weather Condition'].str.contains('few clouds').astype(bool)
dataset['Weather_OvercastClouds'] = dataset['Weather Condition'].str.contains('overcast clouds').astype(bool)
dataset['Weather_ScatteredClouds'] = dataset['Weather Condition'].str.contains('scattered clouds').astype(bool)

### One-hot encoding of the start_neighborhood feature

In [41]:
# One-hot encoding of the start_neighborhood feature
dataset['sn_Verdun'] = dataset['start_neighborhood'].str.contains('Verdun').astype(bool)
dataset['sn_Le Plateau-Mont-Royal'] = dataset['start_neighborhood'].str.contains('Le Plateau-Mont-Royal').astype(bool)
dataset['sn_Ville-Marie'] = dataset['start_neighborhood'].str.contains('Ville-Marie').astype(bool)
dataset['sn_Côte-des-Neiges-Notre-Dame-de-Grâce'] = dataset['start_neighborhood'].str.contains('Côte-des-Neiges-Notre-Dame-de-Grâce').astype(bool)
dataset['sn_Villeray-Saint-Michel-Parc-Extension'] = dataset['start_neighborhood'].str.contains('Villeray-Saint-Michel-Parc-Extension').astype(bool)
dataset['sn_Rosemont-La Petite-Patrie'] = dataset['start_neighborhood'].str.contains('Rosemont-La Petite-Patrie').astype(bool)
dataset['sn_Outremont'] = dataset['start_neighborhood'].str.contains('Outremont').astype(bool)
dataset['sn_Le Sud-Ouest'] = dataset['start_neighborhood'].str.contains('Le Sud-Ouest').astype(bool)
dataset['sn_Mercier-Hochelaga-Maisonneuve'] = dataset['start_neighborhood'].str.contains('Mercier-Hochelaga-Maisonneuve').astype(bool)
dataset['sn_Westmount'] = dataset['start_neighborhood'].str.contains('Westmount').astype(bool)
dataset['sn_Ahuntsic-Cartierville'] = dataset['start_neighborhood'].str.contains('Ahuntsic-Cartierville').astype(bool)
dataset['sn_LaSalle'] = dataset['start_neighborhood'].str.contains('LaSalle').astype(bool)

### One-hot encoding of route_affectations

There are 64 categories, we need to split it in half, at least

**First: Transforming the variable**

In [42]:
dataset['route_affectations'].values

array(['residentiel-residentiel', 'institution-institution',
       'mixte-residentiel', ..., 'mixte-residentiel', 'mixte-parc',
       'residentiel-institution'], dtype=object)

Creating a function to order values (start and end affectations) by alphabetical order, in order to reduce the multiplicity of values to a minimum.

In [43]:
def reorder_route_affectations(value):
    #Splitting the route affectations in two
    affectations = value.split('-')
    #Ordering the list by alphabetical order:
    affectations.sort()
    affectations = '-'.join(affectations)
    return affectations
dataset['route_affectations_ordered'] = dataset['route_affectations'].apply(reorder_route_affectations)

In [44]:
dataset['route_affectations_ordered'] = dataset['route_affectations_ordered'].astype('category')

In [45]:
dataset['route_affectations_ordered'].values

[residentiel-residentiel, institution-institution, mixte-residentiel, mixte-mixte, mixte-mixte, ..., mixte-mixte, residentiel-residentiel, mixte-residentiel, mixte-parc, institution-residentiel]
Length: 4740357
Categories (36, object): [activites diversifiees-activites diversifiees, activites diversifiees-emplois, activites diversifiees-institution, activites diversifiees-mixte, ..., parc-residentiel, religieux-religieux, religieux-residentiel, residentiel-residentiel]

**We have 36 values now, much better!**

**Now, we do the one-hot encoding**

In [46]:
#mixte-residentiel                   value count: 1383263
dataset['ra_mixte-residentiel'] = dataset['route_affectations_ordered'].str.contains("mixte-residentiel").astype(bool)
#residentiel-residentiel                          1037211
dataset['ra_residentiel-residentiel'] = dataset['route_affectations_ordered'].str.contains("residentiel-residentiel").astype(bool)
#mixte-mixte                                       826508
dataset['ra_mixte-mixte'] = dataset['route_affectations_ordered'].str.contains("residentiel-residentiel").astype(bool)
#institution-mixte                                 287514
dataset['ra_institution-mixte'] = dataset['route_affectations_ordered'].str.contains("institution-mixte").astype(bool)
#institution-residentiel                           249713
dataset['ra_institution-residentiel'] = dataset['route_affectations_ordered'].str.contains("institution-residentiel").astype(bool)
#parc-residentiel                                  240312
dataset['ra_parc-residentiel'] = dataset['route_affectations_ordered'].str.contains("parc-residentiel").astype(bool)
#mixte-parc                                        237214
dataset['ra_mixte-parc'] = dataset['route_affectations_ordered'].str.contains("mixte-parc").astype(bool)
#emplois-residentiel                                86356
dataset['ra_emplois-residentiel'] = dataset['route_affectations_ordered'].str.contains("emplois-residentiel").astype(bool)
#parc-parc                                          75962
dataset['ra_parc-parc'] = dataset['route_affectations_ordered'].str.contains("parc-parc").astype(bool)
#mixte-nan                                          46232
dataset['ra_mixte-nan'] = dataset['route_affectations_ordered'].str.contains("mixte-nan").astype(bool)
#institution-parc                                   44424
dataset['ra_institution-parc'] = dataset['route_affectations_ordered'].str.contains("institution-parc").astype(bool)
#emplois-mixte                                      40057
dataset['ra_emplois-mixte'] = dataset['route_affectations_ordered'].str.contains("emplois-mixte").astype(bool)
#activites diversifiees-residentiel                 33751
dataset['ra_activitesdiversifiees-residentiel'] = dataset['route_affectations_ordered'].str.contains("activites diversifiees-residentiel").astype(bool)
#institution-institution                            30830
dataset['ra_institution-institution'] = dataset['route_affectations_ordered'].str.contains("institution-institution").astype(bool)
#nan-residentiel                                    30573
dataset['ra_nan-residentiel'] = dataset['route_affectations_ordered'].str.contains("nan-residentiel").astype(bool)
#activites diversifiees-mixte                       17004
dataset['ra_activitesdiversifiees-mixte'] = dataset['route_affectations_ordered'].str.contains("activites diversifiees-mixte").astype(bool)
#mixte-religieux                                    11954
dataset['ra_mixte-religieux'] = dataset['route_affectations_ordered'].str.contains("mixte-religieux").astype(bool)
#nan-nan                                             9033
dataset['ra_nan-nan'] = dataset['route_affectations_ordered'].str.contains("nan-nan").astype(bool)
#institution-nan                                     8231
dataset['ra_institution-nan'] = dataset['route_affectations_ordered'].str.contains("institution-nan").astype(bool)
#emplois-institution                                 7962
dataset['ra_emplois-institution'] = dataset['route_affectations_ordered'].str.contains("emplois-institution").astype(bool)
#emplois-parc                                        5829
dataset['ra_emplois-parc'] = dataset['route_affectations_ordered'].str.contains("emplois-parc").astype(bool)
#emplois-emplois                                     5545
dataset['ra_emplois-emplois'] = dataset['route_affectations_ordered'].str.contains("emplois-emplois").astype(bool)
#religieux-residentiel                               4371
dataset['ra_religieux-residentiel'] = dataset['route_affectations_ordered'].str.contains("religieux-residentiel").astype(bool)
#activites diversifiees-parc                         3544
dataset['ra_activites diversifiees-parc'] = dataset['route_affectations_ordered'].str.contains("activites diversifiees-parc").astype(bool)
#nan-parc                                            3438
dataset['ra_nan-parc'] = dataset['route_affectations_ordered'].str.contains("nan-parc").astype(bool)
#institution-religieux                               2944
dataset['ra_institution-religieux'] = dataset['route_affectations_ordered'].str.contains("institution-religieux").astype(bool)
#activites diversifiees-institution                  2600
dataset['ra_activitesdiversifiees-institution'] = dataset['route_affectations_ordered'].str.contains("activites diversifiees-institution").astype(bool)
#activites diversifiees-emplois                      2453
dataset['ra_activitesdiversifiees-emplois'] = dataset['route_affectations_ordered'].str.contains("activites diversifiees-emplois").astype(bool)
#nan-religieux                                       1504
dataset['ra_nan-religieux'] = dataset['route_affectations_ordered'].str.contains("nan-religieux").astype(bool)
#activites diversifiees-activites diversifiees       1174
dataset['ra_activitesdiversifiees-activitesdiversifiees'] = dataset['route_affectations_ordered'].str.contains("activites diversifiees-activites diversifiees").astype(bool)
#emplois-nan                                         1160
dataset['ra_emplois-nan'] = dataset['route_affectations_ordered'].str.contains("emplois-nan").astype(bool)
#parc-religieux                                       631
dataset['ra_parc-religieux'] = dataset['route_affectations_ordered'].str.contains("parc-religieux").astype(bool)
#activites diversifiees-nan                           609
dataset['ra_activitesdiversifiees-nan'] = dataset['route_affectations_ordered'].str.contains("activites diversifiees-nan").astype(bool)
#religieux-religieux                                  345
dataset['ra_religieux-religieux'] = dataset['route_affectations_ordered'].str.contains("religieux-religieux").astype(bool)
#activites diversifiees-religieux                      62
dataset['ra_activitesdiversifiees-religieux'] = dataset['route_affectations_ordered'].str.contains("activites diversifiees-religieux").astype(bool)
#emplois-religieux                                     44
dataset['ra_emplois-religieux'] = dataset['route_affectations_ordered'].str.contains("emplois-religieux").astype(bool)

### A second type of one-hot encoding: this time checking if only one of the stations belong to a perticular affectation

In [47]:
dataset['ra2_mixte'] = dataset['route_affectations_ordered'].str.contains("mixte").astype(bool)
dataset['ra2_emplois'] = dataset['route_affectations_ordered'].str.contains("emplois").astype(bool)
dataset['ra2_religieux'] = dataset['route_affectations_ordered'].str.contains("religieux").astype(bool)
dataset['ra2_activitediversifiees'] = dataset['route_affectations_ordered'].str.contains("activite diversifiees").astype(bool)
dataset['ra2_nan'] = dataset['route_affectations_ordered'].str.contains("nan").astype(bool)
dataset['ra2_institution'] = dataset['route_affectations_ordered'].str.contains("institution").astype(bool)
dataset['ra2_parc'] = dataset['route_affectations_ordered'].str.contains("parc").astype(bool)
dataset['ra2_residentiel'] = dataset['route_affectations_ordered'].str.contains("residentiel").astype(bool)

### One-hot encoding of the start_weekday feature

In [48]:
dataset['swd_Monday'] = dataset['start_weekday'] == 0
dataset['swd_Tuesday'] = dataset['start_weekday'] == 1
dataset['swd_Wednesday'] = dataset['start_weekday'] == 2
dataset['swd_Thursday'] = dataset['start_weekday'] == 3
dataset['swd_Friday'] = dataset['start_weekday'] == 4
dataset['swd_Saturday'] = dataset['start_weekday'] == 5
dataset['swd_Sunday'] = dataset['start_weekday'] == 6

### One-hot encoding of the start_month feature

In [49]:
dataset['sm_April'] = dataset['start_month'] == 4
dataset['sm_May'] = dataset['start_month'] == 5
dataset['sm_June'] = dataset['start_month'] == 6
dataset['sm_July'] = dataset['start_month'] == 7
dataset['sm_August'] = dataset['start_month'] == 8
dataset['sm_September'] = dataset['start_month'] == 9
dataset['sm_October'] = dataset['start_month'] == 10

### Changing Data Types of Categorical Variables

In [50]:
dataset['start_code'] = dataset['start_code'].astype('category')
dataset['end_code'] = dataset['end_code'].astype('category')
dataset['Weather Condition'] = dataset['Weather Condition'].astype('category')
dataset['start_neighborhood'] = dataset['start_neighborhood'].astype('category')
dataset['start_affectation'] = dataset['start_affectation'].astype('category')
dataset['end_neighborhood'] = dataset['end_neighborhood'].astype('category')
dataset['end_affectation'] = dataset['end_affectation'].astype('category')
dataset['route_neighborhood'] = dataset['route_neighborhood'].astype('category')
dataset['route_affectations'] = dataset['route_affectations'].astype('category')

In [51]:
dataset['start_code_cat'] = dataset['start_code'].cat.codes
dataset['end_code_cat'] = dataset['end_code'].cat.codes
dataset['Weather Condition_cat'] = dataset['Weather Condition'].cat.codes
dataset['start_neighborhood_cat'] = dataset['start_neighborhood'].cat.codes
dataset['start_affectation_cat'] = dataset['start_affectation'].cat.codes
dataset['end_neighborhood_cat'] = dataset['end_neighborhood'].cat.codes
dataset['end_affectation_cat'] = dataset['end_affectation'].cat.codes
dataset['route_neighborhood_cat'] = dataset['route_neighborhood'].cat.codes
dataset['route_affectations_cat'] = dataset['route_affectations'].cat.codes

### Processing of Missing Values

In [52]:
# Finding out which columns contain missing values
dataset.isna().sum()

start_date                        0
end_date                          0
duration_sec                      0
is_member                         0
start_code                        0
start_name                        0
start_latitude                    0
start_longitude                   0
start_neighborhood             4867
start_great_park            4676037
start_affectation             55051
end_code                          0
end_name                          0
end_latitude                      0
end_longitude                     0
end_neighborhood               5772
end_great_park              4683559
end_affectation               54762
start_date_hour                   0
end_date_hour                     0
Weather Condition                 0
Humidity                          0
Pressure                          0
Wind Speed                        0
Temperature                       0
festivial_name_1            4739964
festival1_lat               4739964
festival1_long              

#### Removing empty values and fixing 'has_festival'

In [53]:
# Replacing missing values with False:'no festival'
dataset['has_festival'].fillna(value=False,inplace=True)

Fixing '0' values for 'has_festival' : turning them into 'False'

In [54]:
def fix_has_festival(value):
    if value == 0 or value == False:
        return False
    elif value == 1 or value == True:
        return True
    else:
        return False
dataset['has_festival'] = dataset['has_festival'].apply(fix_has_festival)

In [55]:
dataset['has_festival'] = dataset['has_festival'].astype(bool)

#### Removing empty values and fixing 'statutory holiday'

In [56]:
# Import festival data - we need to fix statutory holiday
festival = pd.read_csv("Data/Festival/csv_dataset_holiday_festival.csv", parse_dates=['date'], encoding = "gb2312")

In [57]:
festival = festival.filter(axis=1, items=['date','statutory_holiday'])

In [58]:
festival['statutory_holiday'] = festival['statutory_holiday'].astype(bool)

In [59]:
festival['statutory_holiday'].value_counts()

False    1609
True       60
Name: statutory_holiday, dtype: int64

It appears fixed now...

In [60]:
# Merge the festival data with the dataset
dataset = dataset.merge(festival, how='left', left_on='start_date', right_on='date')

In [61]:
dataset.columns

Index(['start_date', 'end_date', 'duration_sec', 'is_member', 'start_code',
       'start_name', 'start_latitude', 'start_longitude', 'start_neighborhood',
       'start_great_park',
       ...
       'end_code_cat', 'Weather Condition_cat', 'start_neighborhood_cat',
       'start_affectation_cat', 'end_neighborhood_cat', 'end_affectation_cat',
       'route_neighborhood_cat', 'route_affectations_cat', 'date',
       'statutory_holiday_y'],
      dtype='object', length=146)

In [62]:
dataset.drop(axis=1,columns=['statutory_holiday_x', 'date'],inplace=True)
dataset.rename({'statutory_holiday_y':'statutory_holiday'}, axis='columns', inplace=True)

In [63]:
dataset.columns

Index(['start_date', 'end_date', 'duration_sec', 'is_member', 'start_code',
       'start_name', 'start_latitude', 'start_longitude', 'start_neighborhood',
       'start_great_park',
       ...
       'start_code_cat', 'end_code_cat', 'Weather Condition_cat',
       'start_neighborhood_cat', 'start_affectation_cat',
       'end_neighborhood_cat', 'end_affectation_cat', 'route_neighborhood_cat',
       'route_affectations_cat', 'statutory_holiday'],
      dtype='object', length=144)

'statutory_holiday' appears fixed as well!

In [64]:
# Removing festival dataset from memory
del festival

In [65]:
dataset.shape

(4740357, 144)

In [66]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4740357 entries, 0 to 4740356
Columns: 144 entries, start_date to statutory_holiday
dtypes: bool(84), category(10), datetime64[ns](4), float64(13), int16(3), int64(12), int8(6), object(12)
memory usage: 2.0+ GB


In [67]:
#Changing category variables back to object, as they cannot be stored this way in HDF format.
dataset['start_code'] = dataset['start_code'].astype('object')
dataset['end_code'] = dataset['end_code'].astype('object')
dataset['Weather Condition'] = dataset['Weather Condition'].astype('object')
dataset['start_neighborhood'] = dataset['start_neighborhood'].astype('object')
dataset['start_affectation'] = dataset['start_affectation'].astype('object')
dataset['end_neighborhood'] = dataset['end_neighborhood'].astype('object')
dataset['end_affectation'] = dataset['end_affectation'].astype('object')
dataset['route_neighborhood'] = dataset['route_neighborhood'].astype('object')
dataset['route_affectations'] = dataset['route_affectations'].astype('object')
dataset['route_affectations_ordered'] = dataset['route_affectations_ordered'].astype('object')

In [68]:
# Storing the modified dataset to disk
dataset.to_hdf("/Users/gabriel/dataset_2017_final.hdf",key="dataset")